# Title - EDA Questions

## Is there a correlation between Studio,Genre and Rating.

## How does series runtime vary over time for tv and movies (analyse seperately)

# Setup

## Modules

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.stats

sns.set_palette("Set3")

## Functions

In [ ]:
def univariate_numerical_analysis(numerical_frame: pd.DataFrame ,column:str, new_index:str):
    
    numerical_data = numerical_frame.loc[:,column]

    mean_val = round(numerical_data.mean(),2)
    median_val = round(numerical_data.median(),2)
    std_val = round(numerical_data.std(),2)
    range_val = round(numerical_data.max() - numerical_data.min(),2)  
    iqr_val = round(numerical_data.quantile(0.75)-numerical_data.quantile(0.25),2)
    skew_val = round(numerical_data.skew(),2)
    kurtosis_val = round(numerical_data.kurtosis(),2)
    coefficient_of_variance_val = round(((std_val / mean_val) if mean_val != 0 else 0),2) 
    mode_val = numerical_data.mode().tolist()    
    mode_val_string = ""

    for _ in mode_val:
        mode_val_string += str(format(_, ".2f")) + ","



    output_frame = pd.DataFrame(
        {"Mean": [mean_val],
         "Median": [median_val],
         "Mode": [mode_val],
         "Standard Deviation": [std_val],
         "Range": [range_val],
         "Inter-Quartile Range": [iqr_val],
         "Skewness": [skew_val],
         "Kurtosis": [kurtosis_val],
         "Coefficient of Variance": [coefficient_of_variance_val]})
    output_frame.index = [new_index]
   
    return(output_frame)

In [ ]:
def iqr_outlier_strip(numerical_frame: pd.DataFrame,column:str):
    
    q1 = np.percentile(numerical_frame[column], 25)
    q3 = np.percentile(numerical_frame[column], 75)
    iqr = q3 - q1
    lower_bound = q1 - 1.5 * iqr
    upper_bound = q3 + 1.5 * iqr

    outlier_mask = (numerical_frame[column] < lower_bound) | (numerical_frame[column] > upper_bound)

    numerical_frame_filtered = numerical_frame[~outlier_mask]

    return(numerical_frame_filtered)

## Loading Data

In [ ]:
df = pd.read_csv("Top_Anime_data.csv")

df.columns = [columns.lower().replace(" ", "_") for columns in df.columns]   ### Anti-whitespace pro snake_case

## Data Inspection

In [ ]:
df.info()

filtering df by data type so data inspectionis easier

In [ ]:
cat = df.select_dtypes(exclude="number")
cat_columns = list(cat.columns)
print(f"The categorical columns of the cat dataframe are :{cat_columns}")

In [ ]:
num = df.select_dtypes(include="number")
num_columns = list(num.columns)
print(f"The numerical columns of num dataframe are :{num_columns}")

### Numeric data check 1

In [ ]:
num_non_null_columns =num.notnull().sum()
num_non_null_total = num.notnull().sum().sum()
num_non_null_percentage_column = num.notnull().sum() / len(num)
num_non_null_percentage_total = (num.notnull().sum().sum()) / (num.size)
                                                                            # Count of non-missing values per column:

num_dup = num.duplicated()
num_dup_total = num.duplicated().sum()                                      # One fewer sum as duplicated returns series not a frame
num_na_columns = num.isna().sum()             
num_na_total = num.isna().sum().sum()

print(f"""
number of non-null values in the frame of numeric data: {num_non_null_total},
number of non-null values by column: \n\n{num_non_null_columns}\n

precentage of non-null values in the frame of numeric data: {num_non_null_percentage_total * 100},
precentage of non-null values by column: \n\n{num_non_null_percentage_column * 100}
      
number of duplicates in the frame of numeric data: {num_dup_total},

number of null values in the frame of numeric data:{num_na_total},
number of null values by column:\n\n{num_na_columns}\n""")

print(f"The following rows are duplicates: {','.join(num[num_dup].index )}") # num_dup is a boolean series which acts as a filter condition
print(f"The following columns have null values: {','.join(num_na_columns[num_na_columns > 0].index)}")


### Categoric data check 1

In [ ]:
##.strip() on cat

stripped = lambda _: _.strip() if isinstance(_, str) else _
cat = cat.map(stripped)

##

cat_non_null_columns = cat.notnull().sum()
cat_non_null_total = cat.notnull().sum().sum()
cat_non_null_percentage_column = cat.notnull().sum() / len(cat)
cat_non_null_percentage_total = (cat.notnull().sum().sum()) / (cat.size)
                                                                            # Count of non-missing values per column:

cat_dup = cat.duplicated()
cat_dup_total = cat.duplicated().sum()                                      # One fewer sum as duplicated returns series not a frame
cat_na_columns = cat.isna().sum()             
cat_na_total = cat.isna().sum().sum()

#cat_spaced_values_total_old = cat.eq(" ").sum().sum()                      # Old basic whitespace detector

cat_spaced_values_columns = cat.astype(str).apply(lambda _ : _.str.match(r'^\s*$')).sum() 
cat_spaced_values_total = cat.astype(str).apply(lambda _ : _.str.match(r'^\s*$')).sum().sum() 

# Treats every column as data type string, and for every column applies the check that it is not continuos whitespace using the regex
# expression r'^\s*$' and the str.match() method. ^ indicates the start, \s*checks for coninuos whitespace, and $indicates the end of the expression. 

print(f"""
number of non-null values in the frame of categoric data: {cat_non_null_total},
number of non-null values by column: \n\n{cat_non_null_columns}

percentage of non-null values in the frame of categoric data: {cat_non_null_percentage_total * 100},
percentage of non-null values by column: \n\n{cat_non_null_percentage_column * 100}

number of duplicates in the frame of categoric data: {cat_dup_total},

number of white space values in the frame of categoric data: {cat_spaced_values_total},
number of white space values by column: \n\n{cat_spaced_values_columns}

number of null values in the frame of categoric data:{cat_na_total},
number of null values by column:\n\n{cat_na_columns}\n""")

print(f"The following rows are duplicates: {','.join(cat[cat_dup].index )}") # cat_dup is a boolean series which acts as a filter condition
print(f"The following columns have null values: {','.join(cat_na_columns[cat_na_columns > 0].index)}")


## Data Formatting

### Making episodes column numeric

All null values set to zero, the zeros can be dropped later if needed.

In [ ]:
df["episodes"] = pd.to_numeric(df["episodes"], errors = "coerce").fillna(0).astype(int)             #null values saved as zero


### Air dates and time on air

In [ ]:
first_last = df["aired"].astype(str).str.split(" to ", n=1, expand=True)        # first_last is a new dataframe that takes each result from the split as a new column


df["aired_first"] = first_last[0].str.strip()
df["aired_last"]  = ( 
    first_last[1]
    .str.strip()
    .replace("?","Dec 31, 2024")                                                #dataset is for anime in 2024 and made in january 2025, Dec 31, 2024 is being used as end date of dataframe
    if first_last.shape[1] > 1 else pd.NA
    )

## .partion() caused so many problems, sometimes fancy speciifc functions are not worth it

# df[["japanese","aired_first","aired_last"]].head(50)                          # Entry 50 one piece is proof this worked

In [ ]:
# ### df["date"] = pd.to_datetime(df["date"], format="%b %d, %Y")

df["aired_first"] = pd.to_datetime(df["aired_first"], format="%b %d, %Y")
df["aired_last"] = pd.to_datetime(df["aired_last"], format="%b %d, %Y")

df["years_on_air"] = (df["aired_last"] - df["aired_first"]).dt.days / 365

# df[["aired_first","aired_last","years_on_air"]].head(50)  

Studios

In [ ]:
## Maybe its better to store studios in list, because we can still sum using indexing, 

df["studios"] = df["studios"].str.split(",").apply(lambda lst: [_.strip() for _ in lst])


# mask = df["col"].apply(lambda x: "b" in x)        #can be used to find indexes thatt contain a studio
# df_filtered = df[mask]


### Splitting genres into genres_list

In [ ]:
tmp = (
    df["genres"]
    .astype(str)
    .str.split(",")
    .explode()                      # new rows for each new unique value 
    .str.strip()                    # hidden whitespace destroyer so string slicing doesn't miscount
    .replace({"nan": pd.NA})        # gets rid of nan, the only "genre" that doesn't repeat so isnt affected the same way by the filter
    .dropna())

tmp = sorted(tmp.unique().tolist())

genres_list = []

for _ in tmp:
    genres_list.append(_[0:(len(_)//2)]) # Removes the duplicated words.

# print(genres_list)

Assigning Genres

In [ ]:
def genre_assignment(dataframe):
    for i in genres_list:
        df[i] = df["genres"].str.contains(i)

genre_assignment(df)

## below is a really dirty fix for removing rows that have no genre as none of the methods were working directly on genres.
## but it works as the only possible values for rows that had anything in the genres column are True of False. NaN reveals an orignally empty genres column.

df = df[~df["Horror"].isna()] 
df.drop(columns = ["genres"])                   #"genres" is no longer needed


### Replacing null English names with Japanese Name.

In [ ]:
df["name"] = df["english"].fillna(df["japanese"])

### moving name to the front of the dataframe

df = df[["name"] + [c for c in df.columns if c != "name"]]

## Data Inspection Again

In [ ]:
# reapplying the snake_case formatting as new columns have been added

df.columns = [columns.lower().replace(" ", "_") for columns in df.columns]   ### Anti-whitespace pro snake_case

In [ ]:
cat = df.select_dtypes(exclude="number")
cat_columns = list(cat.columns)
print(f"The categorical columns of the cat dataframe are :{cat_columns}")


In [ ]:
num = df.select_dtypes(include="number")
num_columns = list(num.columns)
print(f"The numerical columns of num dataframe are :{num_columns}")


### Numeric Data Check again

In [ ]:
num_non_null_columns =num.notnull().sum()
num_non_null_total = num.notnull().sum().sum()
num_non_null_percentage_column = num.notnull().sum() / len(num)
num_non_null_percentage_total = (num.notnull().sum().sum()) / (num.size)
                                                                            # Count of non-missing values per column:

num_dup = num.duplicated()
num_dup_total = num.duplicated().sum()                                      # One fewer sum as duplicated returns series not a frame
num_na_columns = num.isna().sum()             
num_na_total = num.isna().sum().sum()

print(f"""
number of non-null values in the frame of numeric data: {num_non_null_total},
number of non-null values by column: \n\n{num_non_null_columns}\n

precentage of non-null values in the frame of numeric data: {num_non_null_percentage_total * 100},
precentage of non-null values by column: \n\n{num_non_null_percentage_column * 100}
      
number of duplicates in the frame of numeric data: {num_dup_total},

number of null values in the frame of numeric data:{num_na_total},
number of null values by column:\n\n{num_na_columns}\n""")

print(f"The following rows are duplicates: {','.join(num[num_dup].index )}") # num_dup is a boolean series which acts as a filter condition
print(f"The following columns have null values: {','.join(num_na_columns[num_na_columns > 0].index)}")


### Catergoric Data Check again

In [ ]:
##.strip() on cat

stripped = lambda _: _.strip() if isinstance(_, str) else _
cat = cat.map(stripped)

##

cat_non_null_columns = cat.notnull().sum()
cat_non_null_total = cat.notnull().sum().sum()
cat_non_null_percentage_column = cat.notnull().sum() / len(cat)
cat_non_null_percentage_total = (cat.notnull().sum().sum()) / (cat.size)
                                                                            # Count of non-missing values per column:

cat_dup = cat.duplicated()
cat_dup_total = cat.duplicated().sum()                                      # One fewer sum as duplicated returns series not a frame
cat_na_columns = cat.isna().sum()             
cat_na_total = cat.isna().sum().sum()

#cat_spaced_values_total_old = cat.eq(" ").sum().sum()                      # Old basic whitespace detector

cat_spaced_values_columns = cat.astype(str).apply(lambda _ : _.str.match(r'^\s*$')).sum() 
cat_spaced_values_total = cat.astype(str).apply(lambda _ : _.str.match(r'^\s*$')).sum().sum() 

# Treats every column as data type string, and for every column applies the check that it is not continuos whitespace using the regex
# expression r'^\s*$' and the str.match() method. ^ indicates the start, \s*checks for coninuos whitespace, and $indicates the end of the expression. 

print(f"""
number of non-null values in the frame of categoric data: {cat_non_null_total},
number of non-null values by column: \n\n{cat_non_null_columns}

percentage of non-null values in the frame of categoric data: {cat_non_null_percentage_total * 100},
percentage of non-null values by column: \n\n{cat_non_null_percentage_column * 100}

number of duplicates in the frame of categoric data: {cat_dup_total},

number of white space values in the frame of categoric data: {cat_spaced_values_total},
number of white space values by column: \n\n{cat_spaced_values_columns}

number of null values in the frame of categoric data:{cat_na_total},
number of null values by column:\n\n{cat_na_columns}\n""")

print(f"The following rows are duplicates: {','.join(cat[cat_dup].index )}") # cat_dup is a boolean series which acts as a filter condition
print(f"The following columns have null values: {','.join(cat_na_columns[cat_na_columns > 0].index)}")


## Data Cleaning

### Dropping useless columns

In [288]:
columns_to_drop_cat = ["description","synonyms","premiered","broadcast","demographic","japanese","english","producers","licensors","genres","aired"]
cat_cleaned = cat.drop(columns = columns_to_drop_cat)
#display(cat_cleaned)
cat_cleaned.info()

<class 'pandas.core.frame.DataFrame'>
Index: 771 entries, 0 to 998
Data columns (total 28 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   name           770 non-null    object        
 1   type           771 non-null    object        
 2   status         771 non-null    object        
 3   studios        771 non-null    object        
 4   source         771 non-null    object        
 5   duration       771 non-null    object        
 6   rating         771 non-null    object        
 7   aired_first    771 non-null    datetime64[ns]
 8   aired_last     516 non-null    datetime64[ns]
 9   action         771 non-null    bool          
 10  adventure      771 non-null    bool          
 11  avant_garde    771 non-null    bool          
 12  award_winning  771 non-null    bool          
 13  boys_love      771 non-null    bool          
 14  comedy         771 non-null    bool          
 15  drama          771 non-null 

In [289]:
columns_to_drop_num = ["popularity","members"]
num_cleaned = num.drop(columns = columns_to_drop_num)
#display(num_cleaned)
num_cleaned.info()

<class 'pandas.core.frame.DataFrame'>
Index: 771 entries, 0 to 998
Data columns (total 4 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   score         771 non-null    float64
 1   rank          771 non-null    int64  
 2   episodes      771 non-null    int64  
 3   years_on_air  516 non-null    float64
dtypes: float64(2), int64(2)
memory usage: 30.1 KB


# Exploration

## Univariate

## Bivariate

# SQL

 ## Q1

 ## Q2

 ## Q3

 ## Q4

 ## Q5

 ## Q6

 ## Q7

 ## Q8

 ## Q9

 ## Q10